In [1]:
20

20

In [2]:
20

20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [2]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [3]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.model.dropout = 0.5
cfg.params = CN()
cfg.params.alpha = 0.1
cfg.params.diff_importance = 1
cfg.params.csd_importance = 0 #  1e2  # 1e2
cfg.params.mine_lr = 0.01
cfg.params.lam = 0  # 0.01
cfg.params.eps = 1e-20
cfg.params.kld_importance = 0.5  # 0.01
cfg.params.recon_importance = 0.1  # 0.01
cfg.federate.client_num = 5
cfg.params.p = 0.


#### configure other options

In [4]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [5]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [6]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-11 14:05:07,362 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [7]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-11 14:05:08,343 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-11 14:05:08,344 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-11 14:05:08,345 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230111140508


In [8]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_client import LaplacianDomainSeparation1MINEClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINEClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-11 14:05:19,043 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-11 14:05:19,055 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 5
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
  

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-11 14:05:19,230 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 5
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  save_dir: test_dir
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: graph-dt
device: 0
distribute:
  use: False
early_st

rho: 0.0
rho: 0.0
in train
mi_local_global: -0.00148676335811615
mi_global_fixed: -0.0009427070617675781
rec_loss: 0.9967923760414124
kld_loss: 1.8103324174880981
mi_local_global: -0.0007017403841018677
mi_global_fixed: -0.002140447497367859
rec_loss: 1.0010249614715576
kld_loss: 1.800900936126709
mi_local_global: -0.0013845190405845642
mi_global_fixed: -0.001095317304134369
rec_loss: 1.0109741687774658
kld_loss: 1.792757272720337
mi_local_global: -0.000738196074962616
mi_global_fixed: -0.00448029488325119
rec_loss: 1.0080184936523438
kld_loss: 1.7687194347381592
mi_local_global: -0.0002983510494232178
mi_global_fixed: -0.0004329979419708252
rec_loss: 1.0040080547332764
kld_loss: 1.7406845092773438
mi_local_global: -0.002879023551940918
mi_global_fixed: -0.003407523036003113
rec_loss: 0.9991787672042847
kld_loss: 1.7250251770019531
mi_local_global: -0.0005816817283630371
mi_global_fixed: -0.0037430301308631897
rec_loss: 0.9961270093917847
kld_loss: 1.6890103816986084
mi_local_global: -

...

2023-01-11 14:44:21,225 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #5', 'Round': 52, 'Results_raw': {'train_imp_ratio': 1.485117, 'train_loss': 661.792801, 'train_acc': 0.842428, 'train_avg_loss': 0.405759, 'train_total': 1631}}


in train
mi_local_global: 0.021315500140190125
mi_global_fixed: -1.247725009918213
rec_loss: 0.9151919484138489
kld_loss: 1.3943078517913818
mi_local_global: 0.016841590404510498
mi_global_fixed: -1.2754437923431396
rec_loss: 0.9079256653785706
kld_loss: 1.4408621788024902
mi_local_global: 0.01873435080051422
mi_global_fixed: -1.3605152368545532
rec_loss: 0.9120349287986755
kld_loss: 1.4258127212524414
mi_local_global: 0.012563973665237427
mi_global_fixed: -1.442798137664795
rec_loss: 0.9066706895828247
kld_loss: 1.4302942752838135
mi_local_global: 0.029719561338424683
mi_global_fixed: -1.3424928188323975
rec_loss: 0.9072352051734924
kld_loss: 1.4465646743774414
mi_local_global: 0.030302435159683228
mi_global_fixed: -1.3141649961471558
rec_loss: 0.9076988101005554
kld_loss: 1.415654182434082
mi_local_global: 0.02408483624458313
mi_global_fixed: -1.4652447700500488
rec_loss: 0.905336856842041
kld_loss: 1.4563319683074951
mi_local_global: 0.03158533573150635
mi_global_fixed: -1.530115246

2023-01-11 14:44:24,888 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #1', 'Round': 52, 'Results_raw': {'train_imp_ratio': -108.887357, 'train_avg_loss': 1.930746, 'train_total': 901, 'train_loss': 1739.602027}}
2023-01-11 14:44:24,891 (server:480)INFO: {'Role': 'Server #', 'Round': 52, 'Results_avg': {'test_imp_ratio': -1059.43281, 'test_avg_loss': 1.311942, 'test_total': 190.6, 'test_loss': 212.772316, 'val_imp_ratio': -715.372959, 'val_avg_loss': 1.227781, 'val_total': 190.2, 'val_loss': 205.208193, 'test_acc': 0.694014, 'val_acc': 0.693352}}
2023-01-11 14:44:24,891 (monitor:513)INFO: current_best=-88.077032, should_save=False
2023-01-11 14:44:24,892 (monitor:513)INFO: current_best=-88.077032, should_save=False
2023-01-11 14:44:24,975 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 52.
2023-01-11 14:44:24,978 (laplacian_server:167)INFO: ----------- Starting a new training round (Round #53) -------------


in val or test
mi_local_global: 0.009416848421096802
mi_global_fixed: -1.8132339715957642
rec_loss: 0.8888427019119263
kld_loss: 1.4222822189331055
mi_local_global: 0.012560352683067322
mi_global_fixed: -1.7684597969055176
rec_loss: 0.8890232443809509
kld_loss: 1.4597828388214111


KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
round 19: -67

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']